# Handing Missing Data

There are a number of schemes that have been developed to indicate the presence of missing data in a table or DataFrame. Generally, they revolve around one of two strategies: using a **mask** that globally indicates missing values, or choosing a **sentinel value** that indicates a missing entry.

In the masking approach, the mask might be an entirely separate Boolean array, or it may involve appropriation of one bit in the data representation to locally indicate the null status of a value.

In the sentinel approach, the sentinel value could be some data-specific convention, such as indicating a missing integer value with -9999 or some rare bit pattern, or it could be a more global convention, such as indicating a missing floating-point value with NaN (Not a Number), a special value which is part of the IEEE floating-point specification.

## Missing data in Pandas

The way in which Pandas handles missing values is constrained by its reliance on the NumPy package, which does not have a built-in notion of NA values for non-floating-point data types.

Pandas could have followed R's lead in specifying bit patterns for each individual data type to indicate nullness, but this approach turns out to be rather unwieldy. While R contains four basic data types, NumPy supports far more than this: for example, while R has a single integer type, NumPy supports fourteen basic integer types once you account for available precisions, signedness, and endianness of the encoding. Reserving a specific bit pattern in all available NumPy types would lead to an unwieldy amount of overhead in special-casing various operations for various types, likely even requiring a new fork of the NumPy package. Further, for the smaller data types (such as 8-bit integers), sacrificing a bit to use as a mask will significantly reduce the range of values it can represent.

NumPy does have support for masked arrays – that is, arrays that have a separate Boolean mask array attached for marking data as "good" or "bad." Pandas could have derived from this, but the overhead in both storage, computation, and code maintenance makes that an unattractive choice.

With these constraints in mind, Pandas chose to use sentinels for missing data, and further chose to use two already-existing Python null values: the special floating-point NaN value, and the Python None object. This choice has some side effects, as we will see, but in practice ends up being a good compromise in most cases of interest.

### None: Pythonic missing data

**None** is a sentinel value used within arrays with data type 'object'. The use of objects in an array maens that if you perform aggregations like sum() or min() across an array with None value, you will get an error. 

In [5]:
import numpy as np
import pandas as pd

In [8]:
valsl = np.array([1, None, 3, 4])
valsl

array([1, None, 3, 4], dtype=object)

In [9]:
valsl.sum()

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

This reflects that addition between an integer and None is undefined

### NaN: Missing numerical data

The other missing data representation, NaN (Not a Number) is different; it is a special floating-point value recognized by all systems that use the standard IEEE floating-point representation:

In [12]:
vals2 = np.array([1, np.nan, 3, 4])
vals2.dtype

dtype('float64')

Numpy chose a floating type for this array, it means that this array support fast operations. However, the NaN is like a data-virus that affects any other objects it touches. Regardless of the operationsm the result of arithmetic with NaN will be another NaN:

In [14]:
1+np.NaN

nan

In [16]:
0*np.NaN

nan

Note that this means that aggregates over the values are well defined (i.e., they don't result in an error) but not always useful:

In [17]:
vals2.sum(),vals2.min(), vals2.max()

(nan, nan, nan)

The special aggregations tgat will ignore the missing values is as below:

In [20]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(8.0, 1.0, 4.0)

Keep in mind that NaN is specifically a floating-point value and there is no equivalent for integer, strings or other types.

### NaN and None in Pandas

NaN and None both have their places, and Pandas is built to handle the two of them nearly interchangeably, converting between them where appropriate:

In [22]:
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

For types that don't have an available sentinel value, Pandas automatically type-casts when NA values are present. For example, if we set a value in an integer array to np.nan, it will automatically be upcast to a floating-point type to accommodate the NA:

In [24]:
x = pd.Series(range(2), dtype = int)
x

0    0
1    1
dtype: int32

In [25]:
x[0] = None
x

0    NaN
1    1.0
dtype: float64

Notice that in addition to casting the integer array to floating point, Pandas also converts the None to NaN value.

The following table lists the upcasting conventions in Pandas when NA values are introduced:

|Typeclass|	Conversion When Storing NAs	|NA Sentinel Value|
|:---|:---|:---|
|floating|No change|np.nan|
|object	|No change|None or np.nan|
|integer|Cast to float64|np.nan|
|boolean|Cast to object|None or np.nan|

Keep in mind that in Pandas, string data is always stored with an object dtype.

### Operating on Null values

Pandas treats None and NaN as essentially interchangeable for indicating missing or null values. To facilitate this convention, there are several useful methods for detecting, removing, and replacing null values in Pandas data structures. They are:
* isnull(): Generate a boolean mask indicating missing values
* notnull(): Opposite of isnull()
* dropna(): Return a filered version of the data
* fillna(): Return a copy of the data with missing values filled or imputed

#### Detecting null values:

In [28]:
data = pd.Series([1, np.nan, 'hello', None])

In [29]:
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

Mask can be used directly as index:

In [31]:
data[data.notnull()]

0        1
2    hello
dtype: object

#### Dropping null values

In [32]:
data.dropna()

0        1
2    hello
dtype: object

For a DataFrame, there are more options. Consider the following DataFrame:

In [35]:
df = pd.DataFrame([[1, np.nan, 2],
                 [2, 3, 5],
                 [np.nan, 4, 6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


We **cannot drop single values** from a DataFrame; we can only drop full rows or full columns. Depending on the application, you might want one or the other, so dropna() gives a number of options for a DataFrame.

By default, dropna() will drop **all rows** in which any null value is present:

In [36]:
df.dropna()

,0,1,2
1,2.0,3.0,5


Alternatively, you can drop NA values along a different axis; axis=1 drops all columns containing a null value

In [37]:
df.dropna(axis = 1)

,2
0,2
1,5
2,6



But this drops some good data as well; you might rather be interested in dropping rows or columns with all NA values, or a majority of NA values. This can be specified through the **how** or **thresh** parameters, which allow fine control of the number of nulls to allow through.

The default is **how='any'**, such that any row or column (depending on the axis keyword) containing a null value will be dropped. You can also specify **how='all'**, which will only drop rows/columns that are all null values:

In [41]:
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [42]:
df.dropna(axis = 'columns', how = 'all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


For finer-grained control, the thresh parameter lets you specify a minimum number of non-null values for the row/column to be kept:

In [48]:
df.dropna(axis = 'rows', thresh = 3)
# thresh = 3 means that the column should have 3 non NaN calues

,0,1,2,3
1,2.0,3.0,5,NaN


Here the first and last row have been dropped, because they contain only two non-null values.

#### Filling null values

Sometimes rather than dropping NA values, you'd rather replace them with a valid value. This value might be a single number like zero, or it might be some sort of imputation or interpolation from the good values. You could do this in-place using the isnull() method as a mask, but because it is such a common operation Pandas provides the fillna() method, which returns a copy of the array with the null values replaced.

In [49]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

We can fill NA entries with a single value, such as zero:

In [50]:
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

We can specify a forward-fill to propagate the previous value forward:

In [51]:
data.fillna(method = 'ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

Or we can specify a back-fill to propagate the next values backward:

In [52]:
data.fillna(method = 'bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

For DataFrames, the options are similar, but we can also specify an axis along which the fills take place:

In [54]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [57]:
df.fillna(method = 'ffill', axis = 1)

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0


Notice that if a previous value is not available during a forward fill, the NA value remains.